In [60]:
!pip install -qU openai langchain pandas gradio sqlalchemy

In [61]:
from sqlalchemy import MetaData
from sqlalchemy import Column, Integer, String, Table, Date, Float
from sqlalchemy import create_engine
from datetime import datetime
from sqlalchemy import insert

metadata_obj = MetaData()

customers = Table(
    "customers",
    metadata_obj,
    Column("customer_id", Integer, primary_key=True),
    Column("customer_name", String(50), nullable=False),
    Column("customer_address", String(200), nullable=False),
    Column("date_joined", Date, nullable=False),
)
accounts = Table(
    "accounts",
    metadata_obj,
    Column("account_id", Integer, primary_key=True),
    Column("customer_id", Integer),
    Column("account_balance", Float, nullable=False),
    Column("date_opened", Date, nullable=False),
)

engine = create_engine("sqlite:///:memory:")

observations_1 = [
    [1, 'Tom Johns', '14, W close, Watford, WD17 5PP', datetime(1974, 1, 1)],
    [2, 'James Arther', '234, London, LD34 P99', datetime(1950, 1, 2)]
]
observations_2 = [
    [1202034, 1, 5000, datetime(2010, 1, 2)],
    [3456782, 2, 2000, datetime(2020, 1, 2)]
]


def insert_obs(obs):
    stmt = insert(customers).values(
        customer_id=obs[0],
        customer_name=obs[1],
        customer_address=obs[2],
        date_joined=obs[3]
    )
    with engine.begin() as conn:
        conn.execute(stmt)


def insert_obs_acct(obs):
    stmt = insert(accounts).values(
        account_id=obs[0],
        customer_id=obs[1],
        account_balance=obs[2],
        date_opened=obs[3]
    )
    with engine.begin() as conn:
        conn.execute(stmt)


def set_up_db():
    metadata_obj.create_all(engine)
    for obs in observations_1:
        insert_obs(obs)
    for obs in observations_2:
        insert_obs_acct(obs)

In [62]:
set_up_db()

In [69]:
import openai
import warnings
import gradio as gr

from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain.sql_database import SQLDatabase
from langchain.chains import SQLDatabaseChain
from langchain.agents import AgentType

openai.api_key = ""

warnings.filterwarnings('ignore')

# completion llm
llm = OpenAI(
    openai_api_key=openai.api_key,
    temperature=0.0
)

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

sql_tool = Tool(
    name='Banking DB',
    func=sql_chain.run,
    description="Useful for when you need to answer questions about retail banking customers " \
                "and their accounts."
)

tools = load_tools(
    ["llm-math"],
    llm=llm
)

tools.append(sql_tool)

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent='conversational-react-description',
    handle_parsing_errors=True,
    verbose=True,
    max_iterations=5,
    memory=memory
)

In [70]:
print(conversational_agent("What is the account balance for James?"))



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Banking DB
Action Input: Account balance for James

> Entering new SQLDatabaseChain chain...
Account balance for James
SQLQuery:SELECT "account_balance" FROM accounts INNER JOIN customers ON accounts."customer_id" = customers."customer_id" WHERE "customer_name" = 'James Arther'
SQLResult: [(2000.0,)]
Answer:2000.0
> Finished chain.

Observation: 2000.0
Thought: Do I need to use a tool? No
AI: The account balance for James is 2000.0.

> Finished chain.
{'input': 'What is the account balance for James?', 'chat_history': '', 'output': 'The account balance for James is 2000.0.'}


Not working and needs fixing ..

In [ ]:
import gradio as gr

def query_internal_db(query):
    return conversational_agent(query)

db_query_demo_app = gr.Interface(
    fn=query_internal_db,
    inputs=gr.Textbox(lines=1, placeholder="Please Enter the DB query here !"),
    outputs="text",
)
db_query_demo_app.launch(debug=True)